In [1]:
import sys
import json
sys.path.append('..')
sys.path.append('../questions_construction')

from src.common import *
from model_performances import *
from copy import deepcopy

import pandas as pd

EVERYTHING_POSTFIX = 'everything'

In [2]:
def to_df(results_all, answer_type, plan_length,  domain = ALL_DOMAINS_KEY):
    d_ramifications_pretty = {
        WITH_RAMIFICATIONS : 'R',
        WITHOUT_RAMIFICATIONS : 'No R'
    }
    d_answer_type_pretty = {
        TRUE_FALSE_ANSWER : 'T/F',
        FREE_ANSWER : 'Free'
    }
    
    index = []
    data = []
    for question_category in QUESTION_CATEGORIES + [ALL_CATEGORIES_KEY]:
        for ramifications in RAMIFICATION_TYPES:
            index.append((plan_length, question_category, d_ramifications_pretty[ramifications], d_answer_type_pretty[answer_type]))
            data_columns = {}
            for model_name in PROMPT_MODEL_NAMES:
                for prompt_type in PROMPT_TYPES:
                    instance = filter_results(results_all, plan_length, question_category, ramifications, model_name, prompt_type, domain, answer_type)
                    if not len(instance) == 1:
                        raise ValueError(f'len(instance) == {len(instance)}')
                    data_columns[(model_name, prompt_type)] = instance[0][SK_RESULT]
            data.append(data_columns)
    return pd.DataFrame(data, index = index)

def df_to_latex_table(df):
        latex_table = df.to_latex(index=True,
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format)
        return latex_table[latex_table.find('\midrule'):].replace("bottomrule", 'crap').replace("\crap", '').replace("\end{tabular}", '')


In [4]:
answer_type = FREE_ANSWER #TRUE_FALSE_ANSWER
results_all = open_jsonl(os.path.join(STATISTICS_PATH, f'{answer_type}.jsonl'))
# df = to_df(results_all, answer_type, 1)
# df
latex_table_all = '\n'.join([df_to_latex_table(to_df(results_all, answer_type, plan_length)) for plan_length in PLAN_LENGTHS])

/var/folders/_0/bqv6dr454lsgny6_gs_xb0q80000gn/T/ipykernel_1010/3083745180.py:27: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True,
/var/folders/_0/bqv6dr454lsgny6_gs_xb0q80000gn/T/ipykernel_1010/3083745180.py:27: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True,
/var/folders/_0/bqv6dr454lsgny6_gs_xb0q80000gn/T/ipykernel_1010/3083745180.py:27: FutureWarning: In future versions `DataFrame.to_latex` is expect

In [6]:
print(latex_table_all)

\midrule
(1, object\_tracking, R, Free)         &                      0.43 &                  0.80 &                      0.40 &                          0.07 &                      0.03 &                          0.03 &                                    NaN &                               0.08 &                                    NaN \\
(1, object\_tracking, No R, Free)      &                      0.39 &                  0.86 &                      0.80 &                          0.08 &                      0.04 &                          0.04 &                                   0.12 &                               0.14 &                                    NaN \\
(1, fluent\_tracking, R, Free)         &                      0.43 &                  0.72 &                      0.54 &                          0.22 &                      0.26 &                          0.20 &                                    NaN &                               0.25 &                                   

In [15]:
# df.to_latex(index=True,
#                   formatters={"name": str.upper},
#                   float_format="{:.2f}".format)

/var/folders/_0/bqv6dr454lsgny6_gs_xb0q80000gn/T/ipykernel_24924/4010271977.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex(index=True,


'\\begin{tabular}{lrrrrrrrrl}\n\\toprule\n{} &  (gemini, zero\\_shot\\_data) &  (gemini, few\\_shot\\_4) &  (gemini, few\\_shot\\_4\\_cot) &  (llama-2-7b, zero\\_shot\\_data) &  (llama-2-7b, few\\_shot\\_4) &  (llama-2-7b, few\\_shot\\_4\\_cot) &  (mistral\\_7b\\_instruct, zero\\_shot\\_data) &  (mistral\\_7b\\_instruct, few\\_shot\\_4) & (mistral\\_7b\\_instruct, few\\_shot\\_4\\_cot) \\\\\n\\midrule\n(1, object\\_tracking, R, T/F)         &                      0.65 &                  0.69 &                      0.46 &                          0.62 &                      0.31 &                          0.38 &                                    NaN &                               0.54 &                                  None \\\\\n(1, object\\_tracking, No R, T/F)      &                      0.54 &                  0.62 &                      0.75 &                          0.46 &                      0.29 &                          0.62 &                                   0.69 &      

In [14]:
print(df_to_latex_table(df))

\midrule
(1, object\_tracking, R, T/F)         &                      0.65 &                  0.69 &                      0.46 &                          0.62 &                      0.31 &                          0.38 &                                    NaN &                               0.54 &                                  None \\
(1, object\_tracking, No R, T/F)      &                      0.54 &                  0.62 &                      0.75 &                          0.46 &                      0.29 &                          0.62 &                                   0.69 &                               0.65 &                                  None \\
(1, fluent\_tracking, R, T/F)         &                      0.54 &                  0.80 &                      0.80 &                          0.65 &                      0.30 &                          0.47 &                                    NaN &                               0.33 &                                  None \

/var/folders/_0/bqv6dr454lsgny6_gs_xb0q80000gn/T/ipykernel_24924/3083745180.py:27: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True,


In [5]:
print(latex_table_all)

\midrule
(1, object\_tracking, R, T/F)         &                      0.65 &                  0.69 &                      0.46 &                          0.62 &                      0.31 &                          0.38 &                                    NaN &                               0.54 &                                  None \\
(1, object\_tracking, No R, T/F)      &                      0.54 &                  0.62 &                      0.75 &                          0.46 &                      0.29 &                          0.62 &                                   0.69 &                               0.65 &                                  None \\
(1, fluent\_tracking, R, T/F)         &                      0.54 &                  0.80 &                      0.80 &                          0.65 &                      0.30 &                          0.47 &                                    NaN &                               0.33 &                                  None \

In [6]:
latex_table_all

'\\midrule\n(1, object\\_tracking, R, T/F)         &                      0.65 &                  0.69 &                      0.46 &                          0.62 &                      0.31 &                          0.38 &                                    NaN &                               0.54 &                                  None \\\\\n(1, object\\_tracking, No R, T/F)      &                      0.54 &                  0.62 &                      0.75 &                          0.46 &                      0.29 &                          0.62 &                                   0.69 &                               0.65 &                                  None \\\\\n(1, fluent\\_tracking, R, T/F)         &                      0.54 &                  0.80 &                      0.80 &                          0.65 &                      0.30 &                          0.47 &                                    NaN &                               0.33 &                            